In [1]:
!pip install langchain wikipedia langchain-community
!pip install langchain-groq langchain-cohere
!pip install pydantic duckduckgo-search

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=4836aaa01ba141d59f662a6f863c662fab09b480d17d8b05d7fa6273e0865f39
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor

from datetime import datetime
from pydantic import BaseModel

In [3]:
search = DuckDuckGoSearchRun()
search_tool = Tool(name="search", func=search.run, description="Search the web for information, especially the current weather", tool_type="search")

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_characters_max=50)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
def print_time(data: str):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Time now is: {timestamp}\n\n"
################### WRAP THE ABOVE PYTHON FUNCTION AS A TOOL FOR LANGCHAIN, AS BELOW
time_tool = Tool(
    name="print_current_time",
    func=print_time,
    description="Prints the current time.",
)

In [5]:
GROQ_API_KEY = 'YOUR_API_KEY'
COHERE_API_KEY = 'YOUR_API_KEY'

In [6]:
llm = ChatCohere(model='command-a-03-2025',cohere_api_key=COHERE_API_KEY)
llm2 = ChatGroq(model='deepseek-r1-distill-qwen-32b',groq_api_key=GROQ_API_KEY)

In [9]:
quick_response = llm.invoke("Tell me a joke")
print(quick_response)

content="Sure! Here's a joke for you:\n\n**Why don’t skeletons fight each other?**\n\nBecause they don’t have the guts! \n\nHope that brought a smile to your face! 😄" additional_kwargs={'id': '26014206-4a67-45eb-a4a9-90fcd8453a7c', 'finish_reason': 'COMPLETE', 'content': "Sure! Here's a joke for you:\n\n**Why don’t skeletons fight each other?**\n\nBecause they don’t have the guts! \n\nHope that brought a smile to your face! 😄", 'token_count': {'input_tokens': 499.0, 'output_tokens': 47.0}} response_metadata={'id': '26014206-4a67-45eb-a4a9-90fcd8453a7c', 'finish_reason': 'COMPLETE', 'content': "Sure! Here's a joke for you:\n\n**Why don’t skeletons fight each other?**\n\nBecause they don’t have the guts! \n\nHope that brought a smile to your face! 😄", 'token_count': {'input_tokens': 499.0, 'output_tokens': 47.0}} id='run-5ed55426-677d-42b3-8585-66dbac4a7d35-0' usage_metadata={'input_tokens': 499, 'output_tokens': 47, 'total_tokens': 546}


In [11]:
class ResearchResponse(BaseModel):
    summary: str
    topic: str
    tools_used: list[str]
    sources: list[str]

parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You  are a helpful assistant. You must do 3 things in the following order.
            (1) TELL A FROG JOKE,  (2) PRINT THE CURRENT IME by calling the "print_current_time" tool,
            then (3) ANSWER USER QUERY
            Wrap output in this format.\n{format_instructions}
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())
tools = [search_tool, wiki_tool, time_tool]
agent = create_tool_calling_agent(
    llm=llm2,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(agent=agent, tools=tools)
query = input("What can I research for you? ")
raw_response = agent_executor.invoke({"query": query})
#print(raw_response)


try:
    response = parser.parse(raw_response.get('output',''))
    print(response)
    print("Topic only is:", response.topic)
except Exception as e:
    print("Error", e, "Raw response", raw_response)


What can I research for you? Singapore formula 1 winners
Error Invalid json output: The Singapore Grand Prix, held at the Marina Bay Street Circuit, has been a part of the Formula 1 calendar since 2008. Here are the winners of the Singapore Formula 1 race:

- **2008**: Lewis Hamilton
- **2009**: Jenson Button
- **2010**: Sebastian Vettel
- **2011**: Sebastian Vettel
- **2012**: Lewis Hamilton
- **2013**: Sebastian Vettel
- **2014**: Nico Rosberg
- **2015**: Daniel Ricciardo
- **2016**: Nico Rosberg
- **2017**: Max Verstappen
- **2018**: Max Verstappen
- **2019**: Valtteri Bottas
- **2020**: Valtteri Bottas
- **2021**: Carlos Sainz
- **2022**: Lando Norris

These drivers have claimed victory in the Singapore Formula 1 race at various points.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE  Raw response {'query': 'Singapore formula 1 winners', 'output': 'The Singapore Grand Prix, held at the Marina Bay Street Circuit, has been a